In [1]:
import cv2
import os

# Initialize webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Create dataset folder
dataset_path = "dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Ask for user name
user_name = input("Enter your name: ")
user_folder = os.path.join(dataset_path, user_name)

if not os.path.exists(user_folder):
    os.makedirs(user_folder)

print("Capturing images... Press 'q' to quit.")

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        count += 1
        face_img = gray[y:y+h, x:x+w]
        cv2.imwrite(f"{user_folder}/{count}.jpg", face_img)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Capturing Faces", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 100:
        break

cap.release()
cv2.destroyAllWindows()
print(f"Captured {count} images for {user_name}.")

2025-03-29 16:32:13.421 Python[9283:133249] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Capturing images... Press 'q' to quit.


2025-03-29 16:32:19.950 Python[9283:133249] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-29 16:32:19.950 Python[9283:133249] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Captured 100 images for Neev Gupta.


In [2]:
import cv2
import os
import numpy as np
import pickle

# Ensure OpenCV contrib module is installed
if not hasattr(cv2.face, "LBPHFaceRecognizer_create"):
    raise ImportError("OpenCV face module is missing! Install with: pip install opencv-contrib-python")

# Initialize the Face Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Path to dataset folder
dataset_path = "dataset"
faces = []
labels = []
label_dict = {}
current_id = 0

# Process each person's folder in the dataset
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_path):
        continue

    # Assign an ID to each person
    if person_name not in label_dict:
        label_dict[person_name] = current_id
        current_id += 1

    person_id = label_dict[person_name]

    # Process each image in the person's folder
    for image_name in os.listdir(person_path):
        image_path = os.path.join(person_path, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"Skipping invalid image: {image_path}")
            continue

        faces.append(img)
        labels.append(person_id)

# Convert lists to numpy arrays
faces = np.array(faces, dtype="object")
labels = np.array(labels)

# Ensure training data is not empty
if len(faces) == 0:
    raise ValueError("Error: No valid face images found for training!")

print(f"Training on {len(faces)} images from {len(label_dict)} people...")

# Train the recognizer
recognizer.train(faces, labels)

# Save the trained model
recognizer.save("face_model.yml")

# Save label dictionary
with open("labels.pkl", "wb") as f:
    pickle.dump(label_dict, f)

print("Training complete! Model and labels saved.")

Training on 301 images from 3 people...
Training complete! Model and labels saved.


In [ ]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace

# Load trained face recognition model
recognizer = cv2.face.LBPHFaceRecognizer_create()
try:
    recognizer.read("face_model.yml")
    print("Face model loaded successfully!")
except Exception as e:
    print("Error loading face model:", str(e))
    exit()

# Load label dictionary
try:
    with open("labels.pkl", "rb") as f:
        label_dict = pickle.load(f)
    print("Label dictionary loaded:", label_dict)  # Debugging output
except Exception as e:
    print("Error loading labels:", str(e))
    exit()

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not capture frame!")
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]  # Extract face region

        # Recognize the face
        label_id, confidence = recognizer.predict(face_img)
        print(f"Predicted ID: {label_id}, Confidence: {confidence}")  # Debugging output

        # Dynamically adjust threshold based on testing
        ADAPTIVE_THRESHOLD = 60  # You can tune this between 50-70

        # Determine the name
        name = "Unknown"
        if label_id in label_dict.values() and confidence < ADAPTIVE_THRESHOLD:
            # Get name from the label dictionary
            name = [k for k, v in label_dict.items() if v == label_id][0]
        else:
            name = "Unknown"  # If confidence is too high, treat as unknown
        # Detect emotions
        try:
            emotion_analysis = DeepFace.analyze(frame[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
            emotion = emotion_analysis[0]['dominant_emotion']
            print(f"Detected Emotion: {emotion}")
        except Exception as e:
            print("Emotion Detection Error:", str(e))
            emotion = "Unknown"

        # Draw rectangle & text
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"{name} ({emotion})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Display the frame
    cv2.imshow("Face & Emotion Recognition", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

Face model loaded successfully!
Label dictionary loaded: {'Sannidhya': 0, 'Neev Gupta': 1, 'Aakarsh': 2}


2025-03-29 16:33:23.548 Python[9321:134673] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Predicted ID: 1, Confidence: 20.92603341226883
Detected Emotion: neutral
Predicted ID: 1, Confidence: 17.121659958493574
Detected Emotion: neutral
Predicted ID: 1, Confidence: 17.312179103571072
Detected Emotion: neutral
Predicted ID: 1, Confidence: 16.100093485419134
Detected Emotion: neutral
Predicted ID: 1, Confidence: 16.737052208840694


2025-03-29 16:33:25.402 Python[9321:134673] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-29 16:33:25.402 Python[9321:134673] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Detected Emotion: neutral
Predicted ID: 1, Confidence: 17.357383502999415
Detected Emotion: neutral
Predicted ID: 1, Confidence: 15.907493568060424
Detected Emotion: neutral
Predicted ID: 1, Confidence: 16.968422569613168
Detected Emotion: happy
Predicted ID: 1, Confidence: 14.712953489797236
Detected Emotion: happy
Predicted ID: 1, Confidence: 20.090107560222517
Detected Emotion: neutral
Predicted ID: 1, Confidence: 17.486345707798765
Detected Emotion: neutral
Predicted ID: 1, Confidence: 16.932865021127302
Detected Emotion: angry
Predicted ID: 1, Confidence: 16.62844530189072
Detected Emotion: neutral
Predicted ID: 1, Confidence: 15.110492012081547
Detected Emotion: neutral
Predicted ID: 1, Confidence: 14.237551217042698
Detected Emotion: angry
Predicted ID: 1, Confidence: 14.687830726365906
Detected Emotion: neutral
Predicted ID: 1, Confidence: 17.347554290726883
Detected Emotion: angry
Predicted ID: 1, Confidence: 16.722337130017998
Detected Emotion: neutral
Predicted ID: 1, Confid